In [5]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# Replace with your Spotify Developer credentials
SPOTIPY_CLIENT_ID = '65baed18639c4c8badbdcdf6df0d52aa'
SPOTIPY_CLIENT_SECRET = '9a8740069ab548338079f3c2f0659313'

df = pd.read_csv("audio_features_partial2.csv")

print(df)

           Artist                             Album  \
1953         68+1                            unreal   
1954  Sugure Joka  everything will be alright smile   
1955         68+1                            unseen   
1956         68+1                   forgotten dream   
1957     usedcvnt                       ultraviolet   
...           ...                               ...   
8707       NELICK                            PiuPiu   
8708        Népal                     Adios Bahamas   
8709        Népal                             Benji   
8710        Népal                       2016 - 2018   
8711        Népal                           Cheddar   

                                  Song       Date Played  \
1953                            unreal   2024-03-02 7:17   
1954  everything will be alright smile   2024-03-02 7:16   
1955                            unseen   2024-03-02 7:14   
1956                   forgotten dream   2024-03-02 7:08   
1957           143 ways to lose urself 

In [14]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from tqdm import tqdm

SPOTIPY_CLIENT_ID = 'a88e951ced8548528f63a82e95294e25'
SPOTIPY_CLIENT_SECRET = '6629dc1c68f5446c982b6df4a1353805'

df = pd.read_csv("audio_features_partial2.csv")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,
                                                           client_secret=SPOTIPY_CLIENT_SECRET))

consecutive_429 = 0 

def get_audio_features(song_id):
    global consecutive_429
    try:
        audio_features = sp.audio_features([song_id])[0]
        if audio_features is None:
            return None
        # Filter out unwanted keys
        keys_to_exclude = {"id", "analysis_url", "track_href", "type", "uri"}
        filtered_features = {k: v for k, v in audio_features.items() if k not in keys_to_exclude}
        consecutive_429 = 0  # Reset the counter on successful response
        return song_id, filtered_features
    except spotipy.exceptions.SpotifyException as e:
        if e.http_status == 429:
            consecutive_429 += 1
            time.sleep(5)  # Wait before retrying
            if consecutive_429 > 5:
                raise RuntimeError("Too many consecutive 429 responses. Exiting.")
        else:
            consecutive_429 = 0
        return song_id, None

# Function to process rows with rate limiting
def process_row_with_rate_limit(song_id):
    while True:
        try:
            result = get_audio_features(song_id)
            time.sleep(0.4)  # 2.5 requests per second
            return result
        except RuntimeError as e:
            # Stop sending requests and save progress
            df.to_csv('audio_features_partial3.csv', index=False)
            raise e

# Use tqdm to show progress
with tqdm(total=len(df), desc="Processing songs") as pbar:
    for index, row in df.iterrows():
        try:
            if pd.isna(row["danceability"]):  # Check if "danceability" is NaN
                song_id = row["Song ID"]
                song_id, audio_features = process_row_with_rate_limit(song_id)
                if audio_features:
                    df.loc[df['Song ID'] == song_id, audio_features.keys()] = audio_features.values()
            pbar.update(1)
        except Exception as e:
            print(f"Error processing song {row['Song ID']}: {e}")
            pbar.update(1)
            continue

# Save the final dataframe with audio features
df.to_csv("audio_features_partial2.csv", index=False)

print(df)

Processing songs:  13%|█▎        | 901/6759 [00:00<00:03, 1513.05it/s]

Error processing song 4Gyevf6hfKBzh69K79twjP: cannot unpack non-iterable NoneType object


Processing songs: 100%|██████████| 6759/6759 [07:52<00:00, 14.30it/s]  

           Artist                             Album  \
0            68+1                            unreal   
1     Sugure Joka  everything will be alright smile   
2            68+1                            unseen   
3            68+1                   forgotten dream   
4        usedcvnt                       ultraviolet   
...           ...                               ...   
6754       NELICK                            PiuPiu   
6755        Népal                     Adios Bahamas   
6756        Népal                             Benji   
6757        Népal                       2016 - 2018   
6758        Népal                           Cheddar   

                                  Song       Date Played  \
0                               unreal   2024-03-02 7:17   
1     everything will be alright smile   2024-03-02 7:16   
2                               unseen   2024-03-02 7:14   
3                      forgotten dream   2024-03-02 7:08   
4              143 ways to lose urself 

In [15]:
df.to_csv('audio_features_partial2.csv', index=False)

In [19]:
data1 = pd.read_csv("audio_features_partial.csv")
data2 = pd.read_csv("audio_features_partial2.csv")

data = pd.concat([data1, data2])
data = data.drop(columns=['Unnamed: 5', "Date Played"])

data.to_csv('FINAL_SONGS_WITH_AUDIO_FEATURES.csv', index=False)

In [1]:
import pandas as pd

raw_data = pd.read_csv("./temp_csv/arandomguy608.csv")
data = pd.read_csv("FINAL_SONGS_WITH_AUDIO_FEATURES.csv"
                   )
#print(raw_data)
merged_data = pd.merge(raw_data, data, on=['Artist', 'Song'], how='left')
merged_data = merged_data.drop(columns=['Album_y'])
merged_data = merged_data.rename(columns={"Album_x": "Album"})
merged_data.head()


,Artist,Album,Song,Date Played,Song ID,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Drake,Views,One Dance,2024-07-07 23:10,1zi7xx7UVEFkmKfv06H8x0,0.792,0.625,1.0,-5.609,1.0,0.0536,0.00776,0.0018,0.329,0.370,103.967,173987.0,4.0
1,A$AP Rocky,TESTING,Praise the Lord (Da Shine) (feat. Skepta),2024-07-07 23:06,7ycWLEP1GsNjVvcjawXz3z,0.854,0.569,5.0,-8.151,0.0,0.1060,0.06090,0.0816,0.100,0.294,80.015,205040.0,4.0
2,Travis Scott,ASTROWORLD,SICKO MODE,2024-07-07 23:01,2xLMifQCjDGFmkHkpNLD9h,0.834,0.730,8.0,-3.714,1.0,0.2220,0.00513,0.0000,0.124,0.446,155.008,312820.0,4.0
3,Kanye West,Donda,Hurricane,2024-07-07 22:57,6Hfu9sc7jvv6coyy2LlzBF,0.587,0.561,0.0,-6.569,1.0,0.1460,0.04660,0.0000,0.109,0.351,80.009,243187.0,4.0
4,Gunna,WUNNA,DOLLAZ ON MY HEAD (feat. Young Thug),2024-07-07 22:54,3nS9a01VvXHQriLqJYwRqG,0.825,0.458,0.0,-7.470,1.0,0.0577,0.11400,0.0000,0.102,0.161,145.115,197760.0,4.0
